<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# NFTD downsampling and reprojection

_by Jeremy Freeman (CarbonPlan), August 2, 2020_

This notebook downsamples and reprojects NFTD 250m yearly rasters stored in
Cloud Optimized GeoTIFF into 250m and 4000m GeoTIFFs.

**Inputs:**

- COG outputs from `01_nftd_to_cogs.ipynb`

**Outputs:**

- COG outputs after downsampling and reprojection

**Notes:**

- Source CRS and projection extent come from NLCD


In [ ]:
import os

In [ ]:
from carbonplan_data.utils import projections, setup

workdir, upload = setup("jeremy")

In [ ]:
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
import rasterio

dst_profile = cog_profiles.get("deflate")

In [ ]:
def get_files(region, resolution):
    return [
        {
            "source": workdir / f"raw/nftd/{region}_foresttype/250m/raster.tif",
            "target": f"processed/nftd/{region}/{resolution}m/type.tif",
        },
        {
            "source": workdir
            / f"raw/nftd/{region}_forestgroup/250m/raster.tif",
            "target": f"processed/nftd/{region}/{resolution}m/group.tif",
        },
        {
            "source": workdir / f"raw/nftd/{region}_foresttype/250m/error.tif",
            "target": f"processed/nftd/{region}/{resolution}m/type_error.tif",
        },
        {
            "source": workdir / f"raw/nftd/{region}_forestgroup/250m/error.tif",
            "target": f"processed/nftd/{region}/{resolution}m/group_error.tif",
        },
    ]

In [ ]:
for resolution in [250, 4000]:
    for region in ["ak", "conus"]:
        files = get_files(region, resolution)
        crs, extent = projections("albers", region)
        for f in files:
            if "error" in str(f["source"]):
                resampling = "average"
            elif resolution == 4000:
                resampling = "mode"
            else:
                resampling = "near"
            cmd = (
                "gdalwarp "
                "-t_srs '%s' "
                "-te %s "
                "-tr %s %s "
                "-r %s "
                "%s "
                "%s"
            ) % (
                crs,
                extent,
                resolution,
                resolution,
                resampling,
                f["source"],
                "./raster.tif",
            )
            os.system(cmd)
            cog_translate("./raster.tif", "./raster.tif", dst_profile)
            upload("./raster.tif", f["target"])
            os.remove("./raster.tif")

### downsample using thresholding


In [ ]:
groups = [
    100,
    120,
    140,
    160,
    180,
    200,
    220,
    240,
    260,
    280,
    300,
    320,
    340,
    360,
    370,
    380,
    400,
    500,
    600,
    700,
    800,
    900,
    910,
    920,
    940,
    950,
    980,
    990,
]

In [ ]:
for resolution in [4000]:
    for region in ["ak", "conus"]:
        src = rasterio.open(workdir / f"processed/nftd/{region}/250m/group.tif")
        band = src.read(1)
        profile = src.profile
        profile["dtype"] = "uint8"

        for group in groups:
            print(f"region {region} cat {group}")
            crs, extent = projections("albers", region)
            out = (band == group).astype(rasterio.uint8)
            resampling = "average"

            with rasterio.open("./thresholded.tif", "w", **profile) as dst:
                dst.write(out, 1)

            cmd = (
                "gdalwarp "
                "-t_srs '%s' "
                "-te %s "
                "-tr %s %s "
                "-r %s "
                "-ot Float32 "
                "%s "
                "%s"
            ) % (
                crs,
                extent,
                resolution,
                resolution,
                resampling,
                "./thresholded.tif",
                "./raster.tif",
            )

            os.system(cmd)
            cog_translate("./raster.tif", "./raster.tif", dst_profile)
            upload(
                "./raster.tif",
                f"processed/nftd/{region}/{resolution}m/group_g{group}.tif",
            )
            os.remove("./thresholded.tif")
            os.remove("./raster.tif")